<a href="https://colab.research.google.com/github/janghyukjin/MachineLearning/blob/master/Assignment09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images
count2 = 0
#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

def sigmoid(x):
    sigma = 1/(1+np.exp(-1*x))
    return sigma
#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)
list_label2  = np.empty(4000, dtype=int)
list_images = np.empty(0)
list_images2 = np.empty(0)

one_hot_encoding = np.eye(10)
for line in data:
    if count >= 6000:
        line_data   = line.split(',')
        label       = line_data[0]
        im_vector   = np.asfarray(line_data[1:])
        im_vector   = normalize(im_vector)
        list_label2[count2] = label
        list_image[:, count]    = im_vector
        if count == 6000:
            list_images2 = im_vector
        else:
            list_images2 = np.vstack([list_images2, im_vector])
        count += 1
        count2 += 1
        continue
    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)
    list_label[count] = label
    list_image[:, count]    = im_vector
    if count == 0:
        list_images = im_vector
    else:
        list_images = np.vstack([list_images, im_vector])
    count += 1
#
# plot first 150 images out of 10,000 with their labels
#
u = np.random.normal(0,1,785*196).reshape((785,196))

v = np.random.normal(0,1,197*49).reshape((197,49))
w = np.random.normal(0,1,50*10).reshape((50,10))
x = np.empty(0)
y = np.empty(0)
z = np.empty(0)
h = np.empty(0)
bias = np.ones((6000,1))
test_bias = np.ones((4000,1))
m = 6000
test_m = 4000
cost = 0
test_cost = 0
accuracy = 0
test_accuracy = 0
cost_list = np.empty(0)
accuracy_list = np.empty(0)
test_cost_list = np.empty(0)
test_accuracy_list = np.empty(0)
alpha = 0.1
jx = np.empty(0)
cnt = 0
for s in range(1000):
    if s%10 == 0:
        print(s)
    bias_x = np.hstack((bias, list_images))
    y = bias_x.dot(u)
    sig_y = sigmoid(y)
    bias_y = np.hstack((bias, sig_y))
    z = bias_y.dot(v)
    sig_z = sigmoid(z)
    bias_z = np.hstack((bias, sig_z))
    h = bias_z.dot(w)
    sig_h = sigmoid(h)
    cost = 0
    accuracy = 0

    for i in range(6000):
        cost += (1/m)*np.sum(-1*one_hot_encoding[list_label[i]]*np.log(sig_h[i]) - (1-one_hot_encoding[list_label[i]])*np.log(1-sig_h[i])) 
        
    for i in range(6000):
        maximum = np.max(sig_h[i])
        for j in range(10):
            if sig_h[i][j] == maximum:
                if list_label[i] == j:
                    accuracy += 1
                    break
    accuracy /= 6000
    print(accuracy)
    cost_list = np.append(cost_list, cost)
    accuracy_list = np.append(accuracy_list, accuracy)   
    delta_w = np.zeros((50,10))
    delta_v = np.zeros((197,49))
    delta_u = np.zeros((785,196))
    for i in range(6000):
        tmp_delta_w = sig_h[i] - one_hot_encoding[list_label[i]]
        tmp_delta_v = ((np.array([tmp_delta_w])).dot(np.transpose(np.delete(w, (0), axis=0))))*np.array([(sig_z[i])*(1-sig_z[i])])
        tmp_delta_u = ((tmp_delta_v).dot(np.transpose(np.delete(v, (0), axis=0))))*np.array([(sig_y[i])*(1-sig_y[i])]) 
        delta_w += np.transpose(np.array([bias_z[i]])).dot(np.array([tmp_delta_w]))
        delta_v += np.transpose(np.array([bias_y[i]])).dot(tmp_delta_v)
        delta_u += np.transpose(np.array([bias_x[i]])).dot(tmp_delta_u)  
    delta_w /= 6000
    delta_v /= 6000
    delta_u /= 6000
    w -= delta_w*alpha
    v -= delta_v*alpha
    u -= delta_u*alpha
    cnt += 1
    jx = np.append(jx, cnt)

    #test start
    test_bias_x = np.hstack((test_bias, list_images2))
    test_y = test_bias_x.dot(u)
    test_sig_y = sigmoid(test_y)
    test_bias_y = np.hstack((test_bias, test_sig_y))
    test_z = test_bias_y.dot(v)
    test_sig_z = sigmoid(test_z)
    test_bias_z = np.hstack((test_bias, test_sig_z))
    test_h = test_bias_z.dot(w)
    test_sig_h = sigmoid(test_h)
    test_cost = 0
    test_accuracy = 0

    for i in range(4000):
        test_cost += (1/test_m)*np.sum(-1*one_hot_encoding[list_label2[i]]*np.log(test_sig_h[i]) - (1-one_hot_encoding[list_label2[i]])*np.log(1-test_sig_h[i]))

        if list_label2[i] == test_sig_h[i].argmax():
            test_accuracy += 1
    test_accuracy /= 4000
    test_cost_list = np.append(test_cost_list, test_cost)
    test_accuracy_list = np.append(test_accuracy_list, test_accuracy)   



In [0]:
correct_images = []
correct_label = []
incorrect_images = []
incorrect_label = []
correct_cnt = 0
incorrect_cnt = 0
for s in range(1):
    test_bias_x = np.hstack((test_bias, list_images2))
    test_y = test_bias_x.dot(u)
    test_sig_y = sigmoid(test_y)
    test_bias_y = np.hstack((test_bias, test_sig_y))
    test_z = test_bias_y.dot(v)
    test_sig_z = sigmoid(test_z)
    test_bias_z = np.hstack((test_bias, test_sig_z))
    test_h = test_bias_z.dot(w)
    test_sig_h = sigmoid(test_h)
    
    for i in range(4000):
        if list_label2[i] == test_sig_h[i].argmax():
            if correct_cnt<10:
                correct_cnt += 1
                correct_label.append(list_label2[i])
                correct_images.append(list_images2[i])
        else:    
            if incorrect_cnt < 10:
                incorrect_cnt += 1
                incorrect_label.append(test_sig_h[i].argmax())
                incorrect_images.append(list_images2[i])

**Plot the loss curve**

**Plot the accuracy curve**

**Final training accuracy**

**Final testing accuracy**

**Present 10 correctly testing images**

**Present 10 misclassified testing images**